# DLO-JZ Optimiseurs et large batch - Jour 2 

Les cellules dans ce *notebook* ne sont pas prévues pour être modifiées, sauf rares exceptions indiquées dans les commentaires. 

*Notebook rédigé par l'équipe assistance IA de l'IDRIS, octobre 2022*


------------------------

Ce notebook est prévu pour être exécuté à partir d'une machine frontale de Jean-Zay. Le *hostname* doit être jean-zay[1-5].

In [ ]:
!hostname

Un module PyTorch doit avoir été chargé pour le bon fonctionnement de ce Notebook. **Nécessairement**, le module `pytorch-gpu/py3/1.11.0` :

In [ ]:
!module list

Les fonctions *python* de gestion de queue SLURM dévelopées par l'IDRIS et les fonctions dédiées à la formation DLO-JZ sont à importer.

Le module d'environnement pour les *jobs* et la taille des images sont fixés pour ce *notebook*.

**TODO :** choisir un *pseudonyme* (maximum 5 caractères) pour vous différencier dans la queue SLURM et dans les outils collaboratifs pendant la formation et la compétition.

In [ ]:
from idr_pytools import display_slurm_queue, gpu_jobs_submitter, search_log
from dlojz_tools_tp import plot_accuracy, lrfind_plot, plot_accuracy_lr
MODULE = 'pytorch-gpu/py3/1.11.0'
account = 'for@v100'
n_gpu = 2

name = 'pseudo'  #TODO#

------------------------------------

## Dataset et modèle

Pour ce TP, on va utiliser la base de données CIFAR 10 et le modèle Resnet-18 pour pouvoir faire tourner des entrainements complets en un temps raisonnable. Le TP se fera en modifiant le code `cifar10.py`.

### CIFAR 10

#### Train set

In [ ]:
import os
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch
import numpy as np
import matplotlib.pyplot as plt

transform = transforms.Compose([ 
        transforms.RandomHorizontalFlip(),              # Horizontal Flip - Data Augmentation
        transforms.ToTensor()                          # convert the PIL Image to a tensor
        ])
    
    
train_dataset = torchvision.datasets.CIFAR10(root=os.environ['ALL_CCFRSCRATCH']+'/CIFAR_10',
                                             train=True, download=False, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,    
                                           batch_size=4,
                                           shuffle=True)
train_dataset

In [ ]:
batch = next(iter(train_loader))
print('X train batch, shape: {}, data type: {}, Memory usage: {} bytes'
      .format(batch[0].shape, batch[0].dtype, batch[0].element_size()*batch[0].nelement()))
print('Y train batch, shape: {}, data type: {}, Memory usage: {} bytes'
      .format(batch[1].shape, batch[1].dtype, batch[1].element_size()*batch[1].nelement()))

img = batch[0][0].numpy().transpose((1,2,0))
plt.imshow(img)
plt.axis('off')
_ = plt.title('label class: {}'.format(batch[1][0].numpy()))

#### Validation set

In [ ]:
val_transform = transforms.Compose([
                    transforms.ToTensor()                           # convert the PIL Image to a tensor
                    ])
    
val_dataset = torchvision.datasets.CIFAR10(root=os.environ['ALL_CCFRSCRATCH']+'/CIFAR_10',
                                               train=False, download=False, transform=val_transform)
val_dataset

### Resnet-18

In [ ]:
model = models.resnet18()
print('number of total parameters: {}'.format(sum([p.numel() for p in model.parameters()])))
print('number of trainable parameters: {}'.format(sum([p.numel() for p in model.parameters() if p.requires_grad])))

-----------

## Description

Nous étudierons 4 *optimizer* (SGD, AdamW, LAMB et LARS).

A chaque fois nous regarderons le cas d'un apprentissage **Small Batch** et le cas d'un apprentissage **Large Batch**.

 * **Small Batch** : *Global Batch Size* de **256** sur 2 GPU sur **30** *epochs*
 * **Large Batch** : *Global Batch Size* de **8192** sur 2 GPU sur **50** *epochs* 


**Remarque** : 

Le paramètre *wrapped_optimizer* est présent dans le code à cause de l'implémentation de LARS spécifiquement. Car le *LR scheduler* doit prendre en entrée l'*optimizer SGD* de base non *wrapped*. 

Pour les autres *optimizers*, il ne sert à rien. Mais cette astuce permet de basculer sur chaque type d'*optimizer* facilement. 

------------------------------------

## LR Finder

Dans le but d'utiliser un *Cycle Scheduler*, il nous faut d'abord trouver l'intervalle des valeurs du *learning rate* qui auront un effet positif sur l'apprentissage du modèle.

On va lancer le script 'cifar10.py' avec l'option `--findlr` ce qui va lancer l'entrainement sur quelques *epochs* durant lesquelles le *learning rate* va doucement augmenter.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

In [ ]:
display_slurm_queue(name)

In [ ]:
#jobid_sgd_lrf = ['228509', '228510']

Vous pouvez maintenant tracer la courbe de la *loss* en fonction du *learning rate*.

#### Small Batch

In [ ]:
lrfind_plot(jobid_sgd_lrf[:1])

La courbe transparente représente les valeurs réelles, la courbe opaque représente un lissage de ces valeurs.

À partir de cette courbe, vous pouvez trouver les valeurs minimale et maximale acceptables du *learning rate*.

#### Large Batch

In [ ]:
lrfind_plot(jobid_sgd_lrf[1:])

------------------------------------

## TP_opti_0 : *Learning Rate* constant (référence)

On va lancer un entrainement de référence avec un *learning rate* constant.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

**TODO : remplacer XXX par la valeur de *learning rate* choisie**

In [ ]:
display_slurm_queue(name)

In [ ]:
#jobid1 = ['228535']

In [ ]:
jobids=[jobid1]
plot_accuracy_lr(jobids)

## TP_opti_1 : *One Cycle Learning Rate*

Nous allons maintenant modifier le code pour remplacer le *learning rate* constant par un *One Cycle Scheduler* et comparer le résultat avec l'entrainement de référence.

**TODO** : dans le script `cifar10.py`:
* Trouver la ligne de déclaration du *Learning Rate Scheduler* :

```python
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer, factor=1, total_iters=5)
```
* Le remplacer par un *One Cycle Scheduler* 

```python
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=args.lr, steps_per_epoch=N_batch, epochs=args.epochs)
```

__Remarque__ : Le *OneCycleLR* de PyTorch calcule automatiquement une valeur minimale de *learning rate* à partir de la valeur maximale donnée.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

**TODO : remplacer XXX par les valeurs maximales de *learning rate* choisies (Small Batch et Large Batch)**

In [ ]:
display_slurm_queue(name)

In [ ]:
#jobid_sgd = ['228548', '228549']

Vous pouvez comparer les courbes de *test accuracy* et de *learning rate* avec l'entrainement de référence.

In [ ]:
jobids=[jobid1, jobid_sgd[:1]]
plot_accuracy_lr(jobids)

------------------------------------

## TP_opti_2 : Optimiseur *AdamW*

On va maintenant modifier l'optimiseur pour utiliser *AdamW*.

**TODO** : dans le script `cifar10.py`:
* Trouver la ligne de déclaration de l'optimiseur *Stochastic Gradient Descent* :

```python
optimizer = torch.optim.SGD(model.parameters(), args.lr, momentum=args.mom, weight_decay=args.wd)
```
* Le remplacer par l'optimiseur *Adam* :

```python
optimizer = torch.optim.AdamW(model.parameters(), args.lr, betas=(args.mom, 0.999), weight_decay=args.wd)
```

L'optimiseur ayant changé il nous faut recalculer la valeur de *learning rate* maximale à donner en paramètre :

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.


In [ ]:
display_slurm_queue(name)

In [ ]:
#jobid_adam_lrf = ['228561', '228562']

#### Small batch

In [ ]:
lrfind_plot(jobid_adam_lrf[:1])

#### Large batch

In [ ]:
lrfind_plot(jobid_adam_lrf[1:])

Vous pouvez maintenant lancer l'entrainement avec l'optimiseur *AdamW*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

**TODO : remplacer XXX par les valeurs maximales de *learning rate* choisies (Small Batch et Large Batch)**

In [ ]:
display_slurm_queue(name)

In [ ]:
#jobid_adamw = ['228564', '228565']

Vous pouvez comparer les courbes de *test accuracy* et de *train accuracy* avec les entrainements précédents.

#### Small batch

In [ ]:
jobids=[jobid_sgd[:1], jobid_adamw[:1]]
plot_accuracy(jobids)

#### Large batch

In [ ]:
jobids=[jobid_sgd[1:], jobid_adamw[1:]]
plot_accuracy(jobids)

------------------------------------

## TP_opti_3 : Optimiseur *LAMB*

On va maintenant modifier l'optimiseur pour utiliser *LAMB*.

**TODO** : dans le script `cifar10.py`:
* Remplacer l'optimiseur *AdamW* par l'optimiseur *LAMB* :

```python
optimizer = apex.optimizers.FusedLAMB(model.parameters(), args.lr, betas=(args.mom, 0.999), weight_decay=args.wd)
```

Il faut maintenant à nouveau toruver la valeur de *learning rate* maximale à donner en paramètre pour cet optimiseur :

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

In [ ]:
display_slurm_queue(name)

In [ ]:
#jobid_lamb_lrf = ['228608', '228609']

#### Small Batch

In [ ]:
lrfind_plot(jobid_lamb_lrf[:1])

#### Large Batch

In [ ]:
lrfind_plot(jobid_lamb_lrf[1:])

Vous pouvez maintenant lancer l'entrainement avec l'optimiseur LAMB.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

**TODO : remplacer XXX par les valeurs maximales de *learning rate* choisies (Small Batch et Large Batch)**

In [ ]:
display_slurm_queue(name)

In [ ]:
#jobid_lamb = ['228610', '228612']

Vous pouvez comparer les courbes de *test accuracy* et de *train accuracy* avec les entrainements précédents.

#### Small Batch

In [ ]:
jobids=[jobid_sgd[:1], jobid_adamw[:1], jobid_lamb[:1]]
plot_accuracy(jobids)

#### Large Batch

In [ ]:
jobids=[jobid_sgd[1:], jobid_adamw[1:], jobid_lamb[1:]]
plot_accuracy(jobids)

## TP_opti_4 : Optimiseur *LARS*

Pour finir nous allons essayer un entrainement large batch avec l'optimiseur LARS ou LARC (optimisation apex de LARS)

**TODO** : dans le script `cifar10.py`:
* Remplacer l'optimiseur *LAMB* par l'optimiseur *LARC* :

```python
optimizer = ...

wrapped_optimizer = optimizer
```
par

```python
optimizer = torch.optim.SGD(model.parameters(), args.lr, momentum=args.mom, weight_decay=args.wd)
    
wrapped_optimizer = LARC(optimizer)
```

Il faut maintenant à nouveau trouver la valeur de *learning rate* maximale à donner en paramètre pour cet optimiseur :

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

In [ ]:
display_slurm_queue(name)

In [ ]:
#jobid_lars_lrf = ['228624', '228625']

#### Small Batch

In [ ]:
lrfind_plot(jobid_lars_lrf[:1])

#### Large Batch

In [ ]:
lrfind_plot(jobid_lars_lrf[1:])

Vous pouvez maintenant lancer l'entrainement avec l'optimiseur LARS.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

**TODO : remplacer XXX par les valeurs maximales de *learning rate* choisies (Small Batch et Large Batch)**

In [ ]:
display_slurm_queue(name)

In [ ]:
#jobid_lars = ['228631', '228633']

Vous pouvez comparer les courbes de *test accuracy* et de *train accuracy* avec les entrainements précédents.

#### Small Batch

In [ ]:
jobids=[jobid_sgd[:1], jobid_adamw[:1], jobid_lamb[:1], jobid_lars[:1]]
plot_accuracy(jobids)

#### Large Batch

In [ ]:
jobids=[jobid_sgd[1:], jobid_adamw[1:], jobid_lamb[1:], jobid_lars[1:]]
plot_accuracy(jobids)

## &#x2622; ⚠ Bonus expérimental ⚠ &#x2622; 
## TP_opti_5 : Optimiseur *LION* 

Dans cette section bonus, nous allons tester un optimiseur relativement récent : LION.

Il s'agit d'un optimiseur issue de la publication : Symbolic Discovery of Optimization Algorithms, https://arxiv.org/abs/2302.06675

**TODO** : dans le script `cifar10.py`:
* Remplacer l'optimiseur *LARC* par l'optimiseur *LION* :

```python
optimizer = torch.optim.SGD(model.parameters(), args.lr, momentum=args.mom, weight_decay=args.wd)

wrapped_optimizer = LARC(optimizer)
```
par

```python
optimizer = Lion(model.parameters(), lr=args.lr, weight_decay=args.wd)
    
wrapped_optimizer = optimizer
```

Il faut maintenant à nouveau trouver la valeur de *learning rate* maximale à donner en paramètre pour cet optimiseur :

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

In [ ]:
display_slurm_queue(name)

In [ ]:
#jobid_lion_lrf = ['228681', '228682']

#### Small Batch

In [ ]:
lrfind_plot(jobid_lion_lrf[:1])

#### Large Batch

In [ ]:
lrfind_plot(jobid_lion_lrf[1:])

**En complément du *learning rate scheduler*, voici quelques indications des auteurs de l'article :**
> *Based on our experience, a suitable learning rate for Lion is typically 3-10x smaller than that for AdamW. Since the effective weight decay is lr * λ, the value of decoupled weight decay λ used for Lion is 3-10x larger than that for AdamW in order to maintain a similar strength.*

Vous pouvez maintenant lancer l'entrainement avec l'optimiseur LION.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

**TODO : remplacer XXX par les valeurs maximales de *learning rate* choisies (Small Batch et Large Batch). 
Vous devrez aussi définir les valeurs de *weight decay* pour chaque taille de batch.** 

**Pour les relativement petits batchs, vous devriez pouvoir obtenir de bonnes performances en vous servant du learning rate finder et des conseils des auteurs**

**Le cas des larges batchs n'est pas aussi trivial. Lors de nos tests, nous n'avons pas pu identifier des paramètres véritablement plus efficaces.**

**Vous pouvez commencer avec le même *learning rate scheduler* qu'avant mais nous vous invitons vivement à expérimenter des changements dessus.**


**TODO** :  dans le script `cifar10.py`:
* Changer le learning rate scheduler par un CosineAnnealingLR pour lequel le comportement de LION est plus régulier.

```python
scheduler = ...
```
par

```python
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                                       T_max=N_batch*args.epochs, 
                                                       eta_min=args.lr/5) 
```

In [ ]:
display_slurm_queue(name)

In [ ]:
#jobid_lion = ['228705', '228706']

Vous pouvez comparer les courbes de *test accuracy* et de *train accuracy* avec les entrainements précédents.

#### Small Batch

In [ ]:
jobids=[jobid_sgd[:1], jobid_adamw[:1], jobid_lamb[:1], jobid_lars[:1], jobid_lion[:1]]
plot_accuracy(jobids)

#### Large Batch

In [ ]:
jobids=[jobid_sgd[1:], jobid_adamw[1:], jobid_lamb[1:], jobid_lars[1:], jobid_lion[1:]]
plot_accuracy(jobids)

--------------

## Annexe

Avec l'*optimizer* de votre choix (Il faudra modifier le code en fonction) :

Vous pouvez faire d'autres tests en jouant sur les différents paramètres :
* Nombre d'epoch
* La valeur du *weight decay*
* La valeur du *learning rate*
* La taille de batch. Attention : **sur 2 GPU**, donc la taille de *batch* sera **multipliée par 2**.

In [ ]:
n_epoch = 
weight_decay = 
lr = 
batch_size = 
command = f'cifar10.py -b {batch_size} -e {n_epoch} --wd {weight_decay} --lr {lr}'

#### LR finder (Optionnel)

In [ ]:
display_slurm_queue(name)

In [ ]:
#jobid_test_lrf =

In [ ]:
lrfind_plot(jobid_test_lrf)

#### Apprentissage

In [ ]:
display_slurm_queue(name)

In [ ]:
#jobid_test = ['428']

In [ ]:
plot_accuracy(jobid_test)

--------------